In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#----------------------------
# Simulation Parameters
#----------------------------
Lx = 2.0       # Domain length in x-direction
Ly = 1.0       # Domain length in y-direction
Nx = 32        # Number of grid points in x
Ny = 32        # Number of grid points in y

Re = 100.0     # Reynolds number
U_bulk = 1.0   # Target bulk velocity

nu = U_bulk * Ly / Re    # Kinematic viscosity

dx = Lx / (Nx - 1)    # Grid spacing in x
dy = Ly / (Ny - 1)    # Grid spacing in y

dt = 0.001           # Time step size
n_steps = 5000       # Number of time steps
output_interval = 500  # Interval for plotting

# Pressure gradient driving the flow
# pressure_gradient = -1.0   # Negative x-direction force
pressure_gradient = 1.0   # Negative x-direction force

# SOR parameters
sor_omega = 1.7      # Over-relaxation factor
sor_tolerance = 1e-4 # Convergence tolerance
sor_max_iter = 5000  # Maximum number of iterations

#----------------------------
# Initialize Fields
#----------------------------
u = np.zeros((Nx, Ny))  # x-velocity
v = np.zeros((Nx, Ny))  # y-velocity
p = np.zeros((Nx, Ny))  # pressure field

# Add random perturbations to the initial velocity field
np.random.seed(0)  # For reproducibility
# perturbation_strength = 0.05
perturbation_strength = 0.0
u += perturbation_strength * (np.random.rand(Nx, Ny) - 0.5)
v += perturbation_strength * (np.random.rand(Nx, Ny) - 0.5)

#----------------------------
# Helper functions
#----------------------------

def build_rhs(u, v, dx, dy, dt):
    # Build RHS of Pressure Poisson Equation based on continuity
    dudx = (u[2:,1:-1] - u[:-2,1:-1]) / (2*dx)
    dvdy = (v[1:-1,2:] - v[1:-1,:-2]) / (2*dy)
    rhs = (dudx + dvdy) / dt
    return rhs

def sor_poisson(p, rhs, dx, dy, omega, tol, max_iter):
    # Solve Pressure Poisson Equation using SOR
    pn = np.copy(p)
    dx2 = dx*dx
    dy2 = dy*dy
    beta = dx2 * dy2 / (2 * (dx2 + dy2))

    for it in range(max_iter):
        p_old = pn.copy()
        pn[1:-1,1:-1] = (1-omega)*pn[1:-1,1:-1] + (omega/2)*( (p_old[2:,1:-1] + p_old[:-2,1:-1])*dy2 + (p_old[1:-1,2:] + p_old[1:-1,:-2])*dx2 - rhs*dx2*dy2 ) / (dx2 + dy2)

        # Enforce Neumann boundary conditions (dp/dn = 0)
        pn[0,:] = pn[1,:]    # Left
        pn[-1,:] = pn[-2,:]  # Right
        pn[:,0] = pn[:,1]    # Bottom
        pn[:,-1] = pn[:,-2]  # Top

        # Check convergence
        res = np.linalg.norm(pn - p_old, ord=2)
        if res < tol:
            break
    return pn

#----------------------------
# Main Time Advancement Loop
#----------------------------

# For plotting
y_center = np.linspace(0, Ly, Ny)

# ここでFigureを準備（最初に一度だけ）
plt.figure(figsize=(8,6))
plt.xlabel("Streamwise Velocity u")
plt.ylabel("Height y")
plt.title("Development of Velocity Profile in Channel Flow")
plt.grid()
# plt.xlim(0, U_bulk * 2)  # x軸のスケールを固定
plt.xlim(0, 5)  # x軸のスケールを固定
plt.ylim(0, Ly)          # y軸のスケールを固定

for n in range(n_steps):

    # Store old velocities
    u_old = u.copy()
    v_old = v.copy()

    # Predictor Step: Explicit Euler update for tentative velocity (u*, v*)

    # Compute derivatives
    dudx = (u[2:,1:-1] - u[:-2,1:-1]) / (2*dx)
    dudy = (u[1:-1,2:] - u[1:-1,:-2]) / (2*dy)
    dvdx = (v[2:,1:-1] - v[:-2,1:-1]) / (2*dx)
    dvdy = (v[1:-1,2:] - v[1:-1,:-2]) / (2*dy)

    lapu = (u[2:,1:-1] - 2*u[1:-1,1:-1] + u[:-2,1:-1])/dx**2 + (u[1:-1,2:] - 2*u[1:-1,1:-1] + u[1:-1,:-2])/dy**2
    lapv = (v[2:,1:-1] - 2*v[1:-1,1:-1] + v[:-2,1:-1])/dx**2 + (v[1:-1,2:] - 2*v[1:-1,1:-1] + v[1:-1,:-2])/dy**2

    # Update tentative velocities
    u[1:-1,1:-1] = u[1:-1,1:-1] + dt*(-(u[1:-1,1:-1]*dudx + v[1:-1,1:-1]*dudy) + nu*lapu + pressure_gradient)
    v[1:-1,1:-1] = v[1:-1,1:-1] + dt*(-(u[1:-1,1:-1]*dvdx + v[1:-1,1:-1]*dvdy) + nu*lapv)

    # Enforce no-slip at walls (top and bottom)
    u[:,0] = 0
    u[:,-1] = 0
    v[:,0] = 0
    v[:,-1] = 0

    # Enforce periodic boundary in x
    u[0,:] = u[-2,:]
    u[-1,:] = u[1,:]
    v[0,:] = v[-2,:]
    v[-1,:] = v[1,:]

    # Corrector Step: Solve for pressure and correct velocity
    rhs = build_rhs(u, v, dx, dy, dt)
    p = sor_poisson(p, rhs, dx, dy, sor_omega, sor_tolerance, sor_max_iter)

    # Correct velocity
    dpdx = (p[2:,1:-1] - p[:-2,1:-1]) / (2*dx)
    dpdy = (p[1:-1,2:] - p[1:-1,:-2]) / (2*dy)

    u[1:-1,1:-1] -= dt * dpdx
    v[1:-1,1:-1] -= dt * dpdy

    # Again enforce boundaries
    u[:,0] = 0
    u[:,-1] = 0
    v[:,0] = 0
    v[:,-1] = 0

    u[0,:] = u[-2,:]
    u[-1,:] = u[1,:]
    v[0,:] = v[-2,:]
    v[-1,:] = v[1,:]

#----------------------------
# Plot results
#----------------------------

    if n % output_interval == 0:
        u_centerline = u[Nx//2,:]
        plt.plot(u_centerline, y_center, label=f"t={n*dt:.2f}")
        print(f"Step {n}/{n_steps} completed.")

# 全ステップ終了後、凡例を付ける
plt.legend()
plt.show()